### SPECT/CT + Sestamibi Sensitivity

- Applied custom concordance logic to classify scan outcomes.
- Counted true positives: `Yes (Exact)` or `Yes (Side)`.
- Counted false negatives: `Non-localizing` scans.
- Calculated sensitivity:  
  **Sensitivity = TP / (TP + FN)**

In [1]:
import pandas as pd 
# preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [9]:
def check_concordance(row):
    imaging = row['SPECT/CT-Sestamibi']
    surgical = row['Surgical Findings']

    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Clean imaging string
    imaging_clean = imaging.replace('SPECT/CT:', '').replace('Sestamibi:', '').replace('|', ',').strip()

    if imaging_clean == 'No SPECT/CT or Sestamibi Recorded':
        return 'Unknown'

    # Hyperplasia always Incorrect Localization
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    if imaging_clean == 'Non-localizing':
        return 'Non-localizing'

    imaging_glands = [g.strip().lower() for g in imaging_clean.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]

    imaging_set = set(imaging_glands)
    surgical_set = set(surgical_glands)

    if surgical_set.issubset(imaging_set):
        return 'Yes (Exact)'
    elif imaging_set & surgical_set:
        return 'Yes (Partial)'

    # Side match check
    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]

    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'

    return 'Incorrect Localization'

In [13]:
# Apply concordance function
preop_imaging_df['Calculated Concordance'] = preop_imaging_df.apply(check_concordance, axis=1)

# Filter for localized scans
localized_spectmibi = preop_imaging_df[
    preop_imaging_df['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)'])
].copy()

nonlocalized = preop_imaging_df[
    preop_imaging_df['Calculated Concordance'] == 'Non-localizing'
].copy()

# Sensitivity calculation
true_positives = localized_spectmibi['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()
false_negatives = nonlocalized.shape[0]
sensitivity = true_positives / (true_positives + false_negatives)

print(f"SPECT/CT + Sestamibi Sensitivity: {sensitivity:.2%} ({true_positives} TP / {true_positives + false_negatives} total)")

SPECT/CT + Sestamibi Sensitivity: 72.78% (254 TP / 349 total)
